In [81]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import SGD
from keras.utils.np_utils import to_categorical  

# (training_data, training_labels), (testing_data, testing_labels) = (cifar100.load_data("coarse"))

# centre the data
def centre_data(train, validation, test):
    
    # calculate the means for each attribute of the training data
    column_means = np.mean(train, axis=0) 
    
    # centre training data by subtracting training data attribute means
    for i in range(len(train)):
        train[i] = train[i] - column_means
    
    # centre testing data by subtracting training data attribute means
    for x in range(len(test)):
        test[x] = test[x] - column_means
        
    for x in range(len(validation)):
        validation[x] = validation[x] - column_means
        
    return train, test, validation

# apply PCA on the data 
def PCA(variance_target, training_data, validation_data, testing_data):

    U, sigma, Vt = np.linalg.svd(training_data, full_matrices=False)
    
    sum_square_singular = np.sum(sigma**2)
    
    ratios = sigma**2/sum_square_singular
    
                
    n_components = 0
    explained_variance = 0
    
    # determine how many principle components must be retained to maintain the target level of explained variance
    for i in range(len(ratios)):
        if explained_variance >= variance_target:
            break
        else: 
            n_components += 1
            explained_variance += ratios[i]
    
    return training_data.dot(Vt.T[:, :n_components]), testing_data.dot(Vt.T[:, :n_components]), validation_data.dot(Vt.T[:, :n_components])

def increase_image_contrast(image):
    xp = [0, 64, 128, 192, 255]
    fp = [0, 16, 128, 240, 255]
    x = np.arange(256)
    table = np.interp(x, xp, fp).astype('uint8')
    return cv2.LUT(image, table)

def increase_all_contrast(train, test):
    new_train = np.copy(train)
    new_test = np.copy(test)
    for i in range(len(train)):
        new_train[i] = increase_image_contrast(train[i])
    for i in range(len(test)):
        new_test[i] = increase_image_contrast(test[i])
    return new_train, new_test

(training_data, training_labels), (testing_data, testing_labels) = (cifar100.load_data("coarse"))


## put in preprocessing things here
## --------------------------------

## --------------------------------

training_data = np.reshape(training_data,(50000, 3072))
testing_data = np.reshape(testing_data,(10000, 3072))

validation_data = training_data[49000:, :]
validation_labels = np.squeeze(training_labels[49000:, :])

training_data = training_data[:49000, :]
training_labels = np.squeeze(training_labels[:49000, :])

training_data = training_data.astype('float32')
testing_data = testing_data.astype('float32')
validation_data = validation_data.astype('float32')

training_data, testing_data, validation_data = centre_data(training_data, validation_data, testing_data)

training_data, testing_data, validation_data = PCA(0.8, training_data, validation_data, testing_data)

training_labels = to_categorical(training_labels, 20)
testing_labels = to_categorical(testing_labels, 20)

# Normalization of pixel values (to [0-1] range)
training_data = training_data / 255
testing_data = testing_data / 255

In [82]:
# Normalization of pixel values (to [0-1] range)

model = Sequential()

model.add(Dense(256, activation='elu', input_dim=training_data.shape[1]))

model.add(Dense(256, activation='elu'))

model.add(Dense(20, activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(optimizer=sgd,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(training_data, training_labels, epochs=15, batch_size=32, verbose=2, validation_split=0.2)

score = model.evaluate(testing_data, testing_labels, batch_size=128, verbose=0)

print(model.metrics_names)
print(score)

Epoch 1/15
1225/1225 - 5s - loss: 2.4661 - accuracy: 0.2450 - val_loss: 2.3509 - val_accuracy: 0.2826
Epoch 2/15
1225/1225 - 3s - loss: 2.3026 - accuracy: 0.2915 - val_loss: 2.2943 - val_accuracy: 0.2968
Epoch 3/15
1225/1225 - 3s - loss: 2.2518 - accuracy: 0.3047 - val_loss: 2.3128 - val_accuracy: 0.2863
Epoch 4/15
1225/1225 - 3s - loss: 2.2166 - accuracy: 0.3168 - val_loss: 2.2642 - val_accuracy: 0.3082
Epoch 5/15
1225/1225 - 3s - loss: 2.1879 - accuracy: 0.3288 - val_loss: 2.2581 - val_accuracy: 0.3083
Epoch 6/15
1225/1225 - 3s - loss: 2.1649 - accuracy: 0.3341 - val_loss: 2.2249 - val_accuracy: 0.3235
Epoch 7/15
1225/1225 - 3s - loss: 2.1414 - accuracy: 0.3404 - val_loss: 2.2233 - val_accuracy: 0.3229
Epoch 8/15
1225/1225 - 3s - loss: 2.1230 - accuracy: 0.3466 - val_loss: 2.2308 - val_accuracy: 0.3295
Epoch 9/15
1225/1225 - 5s - loss: 2.1051 - accuracy: 0.3522 - val_loss: 2.2484 - val_accuracy: 0.3162
Epoch 10/15
1225/1225 - 5s - loss: 2.0855 - accuracy: 0.3579 - val_loss: 2.2449 - 